In [1]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from heapy.util.time import fermi_utc_to_met
from heapy.data.retrieve import gbmRetrieve
from heapy.data.geometry import gbmGeometry
from heapy.data.event import gbmTTE

In [2]:
utc = '2024-02-19T06:21:42'
fermi_met = fermi_utc_to_met(utc)

In [3]:
gbm_rtv = gbmRetrieve.from_utc(utc=utc, t1=-1800, t2=1800)

In [4]:
gbm_geo = gbmGeometry(file=gbm_rtv.rtv_res['poshist'])

for det in ['n9', 'na']:
    gbm_tte = gbmTTE(gbm_rtv.rtv_res['tte'][det])
    gbm_tte.timezero = fermi_met
    gbm_tte.filter_time([-1800, 1800])
    gbm_tte.filter_energy([10, 1000])    
    gbm_tte.lc_binsize = 5

    gbm_saa = gbm_geo.saa_passage(met=gbm_tte.lc_time+gbm_tte.timezero)
    gbm_sun = gbm_geo.sun_visible(met=gbm_tte.lc_time+gbm_tte.timezero)
    gbm_loc = gbm_geo.location_visible(ra=80.016, dec=25.541, met=gbm_tte.lc_time+gbm_tte.timezero)
    gbm_ang = gbm_geo.detector_angle(ra=80.016, dec=25.541, det=det, met=gbm_tte.lc_time+gbm_tte.timezero)
    
    fig = make_subplots(
        rows=5, cols=1,
        row_heights=[0.2, 0.2, 0.2, 0.2, 0.2], 
        shared_xaxes=True,
        horizontal_spacing=0,
        vertical_spacing=0.02)
    
    lc = go.Scatter(x=gbm_tte.lc_time, 
                    y=gbm_tte.lc_src_rate, 
                    mode='lines+markers', 
                    name='lightcurve', 
                    showlegend=True, 
                    error_y=dict(
                        type='data',
                        array=gbm_tte.lc_src_rate_err,
                        thickness=1.5,
                        width=0), 
                    marker=dict(symbol='cross-thin', size=0))
    fig.add_trace(lc, row=1, col=1)
    
    saa = go.Scatter(x=gbm_tte.lc_time, 
                     y=gbm_saa.astype(int), 
                     mode='lines', 
                     name='SAA passage', 
                     showlegend=True)
    fig.add_trace(saa, row=2, col=1)
    
    sun = go.Scatter(x=gbm_tte.lc_time, 
                     y=gbm_sun.astype(int), 
                     mode='lines', 
                     name='sun visible', 
                     showlegend=True)
    fig.add_trace(sun, row=3, col=1)
    
    loc = go.Scatter(x=gbm_tte.lc_time, 
                     y=gbm_loc.astype(int), 
                     mode='lines', 
                     name='location visible', 
                     showlegend=True)
    fig.add_trace(loc, row=4, col=1)
    
    ang = go.Scatter(x=gbm_tte.lc_time, 
                     y=gbm_ang, 
                     mode='lines', 
                     name='detector angle', 
                     showlegend=True)
    fig.add_trace(ang, row=5, col=1)
    
    fig.update_xaxes(title_text=f'Time since {gbm_tte.timezero_utc} (s)', row=5, col=1)
    fig.update_yaxes(title_text=f'Rate', row=1, col=1)
    fig.update_yaxes(title_text=f'Bool', row=2, col=1)
    fig.update_yaxes(title_text=f'Bool', row=3, col=1)
    fig.update_yaxes(title_text=f'Bool', row=4, col=1)
    fig.update_yaxes(title_text=f'Angle', row=5, col=1)
    fig.update_layout(template='plotly_white', height=600, width=800)
    fig.update_layout(legend=dict(x=1, y=1, xanchor='right', yanchor='bottom'))
    
    fig.show()